In [8]:
#Analyze temperature trends and visualize the results:
## Data Preparation - Use NumPy to generate a synthetic dataset representing average monthly temperatures (in degrees Celsius) for 12 months across 10 different cities. The temperatures should range from -5 to 35 degrees.
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True, precision=0)  #removes decimals
data = np.random.uniform(-5, 35, 120) # generate the temperature data

updated = data.reshape(10, 12) #10 cities and 12 months structure

print(updated)


cities = np.array(["New York", "London", "Copenhagen", "Beijing", "Buenos Aires", "Istanbul", "Auckland", "Lima", "Moscow", "Hanoi"])

randcity = np.random.choice(cities, size=10, replace=False)
months = np.array(["Jan","Feb","Mar","Apr","May","Jun",
          "Jul","Aug","Sep","Oct","Nov","Dec"])

df = pd.DataFrame(updated, index=randcity, columns=months) # Convert this NumPy array into a Pandas DataFrame, adding city names as index and months as columns.

print(df)



[[21. 23. 24. 16.  5.  8. 25. -0. 20. 18. 10.  5.]
 [23. 32. 32. 23. 15. 28.  6. 12. 29.  9. 19. -5.]
 [32. 14. -2. 30. -2. 25.  6. 14. 17. 12.  2. 23.]
 [ 7.  5. 10.  7. 11. 25. 30.  8. 17. 10. 24. 27.]
 [11. 12. 25. 21. 16.  4.  9. 25. 30. 29. 25. 19.]
 [24.  3. 12. 15. 23.  7. 31. 20. 29. 21.  5. 15.]
 [10. 33. 34. 32. 17. 25. 27. 23.  9. -4. 17.  5.]
 [23.  7. 28.  8. 31. 14. 11. -1.  4. 12. 15. 23.]
 [ 9. 17. 17.  1. 22. 27. 15. 17. 23. 33. 31. -5.]
 [-3. 11. 19. 11. 13. -1. 13.  9. 27. 24. 34. 21.]]
                    Jan        Feb        Mar        Apr        May  \
Buenos Aires  20.966155  22.937895  23.702226  15.571162   5.033258   
Moscow        23.343453  31.745903  32.061694  23.107204  15.178019   
Beijing       32.184225  13.986742  -2.366257  30.135327  -2.357689   
New York       7.364053   4.813030  10.163704   6.633847  10.655985   
Lima          11.190220  12.206254  25.053348  21.483618  16.497839   
Copenhagen    24.476352   2.635935  11.812346  14.715368  22.92

In [32]:
#Data Analysis:
df.shape
#Calculate the annual average temperature for each city.
avgtemp = df.mean(axis=1) #formula

for city, value in avgtemp.items():
    print(city, ":", int(round(value)),"C")

citymax = avgtemp.idxmax()

print(f"The city with the highest average temperature is: {citymax}") #find the city with the highest average temp

citymin = avgtemp.idxmin()
print(f"The city with the lowest average temperature is: {citymin}") #find the city with the lowest average temp






Buenos Aires : 14 C
Moscow : 19 C
Beijing : 14 C
New York : 15 C
Lima : 19 C
Copenhagen : 17 C
Hanoi : 19 C
Auckland : 15 C
London : 17 C
Istanbul : 15 C
The city with the highest average temperature is: Hanoi
The city with the lowest average temperature is: Beijing


In [38]:
df_reset = df.reset_index() #need to adjust df so that i have specific column names
df_reset.head()


dfname = pd.melt(
    df_reset,
    id_vars=["index"],            # keep the city column
    var_name="Month",             # old column names (Jan, Feb, …)
    value_name="Temperature"      # values
)


dfname = dfname.rename(columns={"index": "City"})

dfname.head() #confirming that it is updated





,City,Month,Temperature
0,Buenos Aires,Jan,20.966155
1,Moscow,Jan,23.343453
2,Beijing,Jan,32.184225
3,New York,Jan,7.364053
4,Lima,Jan,11.190220


In [42]:
#Data Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = px.line(
    dfname,
    x="Month", y="Temperature", color="City",
    markers=True,
    title="Monthly Temperature Trends by City")
fig.show() #testing the first image, i think it's too busy, so now I will change it


multifig = px.line(dfname, x="Month", y="Temperature", color="City",
              facet_col="City", facet_col_wrap=5)
multifig.show()


In [76]:
import plotly.graph_objects as go
import numpy as np

# 1) Compute mins, align to the same city order as maxtemp
mintemp = dfname.groupby("City")["Temperature"].min().reset_index()
mintemp = mintemp.set_index("City").reindex(maxtemp["City"]).reset_index()

# 2) Add as a negative bar trace (left side)
fig.add_trace(
    go.Bar(
        x=-mintemp["Temperature"],           # negate → bars extend left
        y=maxtemp["City"],
        orientation="h",
        name="Min",
        marker_color="royalblue",
        text=mintemp["Temperature"],         # show the *actual* min value
        texttemplate="%{text:.1f}°C",
        textposition="inside",
        insidetextanchor="end"               # at the far end of the bar
    )
)

# 3) Keep a nice symmetric x-axis around zero
mx = max(maxtemp["Temperature"].max(), mintemp["Temperature"].max())
fig.update_xaxes(range=[-mx*1.15, mx*1.15], zeroline=True, zerolinewidth=2)

# 4) Optional: tidy legend / layout
fig.update_layout(barmode="overlay", legend_title_text="Extreme")
fig.show()
